In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

import warnings
warnings.filterwarnings('ignore')

## Uploading the data

In [19]:
def upload_data(data):
    spontan_list= [] 
    sync_list= []
    alone_list = []
    for participant in listdir(complete_data):
        for file in listdir(complete_data+participant+'/'):
            if file.endswith('Spontan.csv'):
                spontan=pd.read_csv(complete_data+participant+'/'+file)
            elif file.endswith('Sync.csv'):
                sync=pd.read_csv(complete_data+participant+'/'+file)
            elif file.endswith('Alone.csv'):
                alone=pd.read_csv(complete_data+participant+'/'+file)
        spontan_list.append(spontan)
        sync_list.append(sync)
        alone_list.append(alone)
    return spontan_list,sync_list,alone_list

In [95]:
#train set
complete_data=(r'C:\Users\meirk\Desktop\final_project_files\extraFiles\Training/')
spontan_list, sync_list ,alone_list = spontan_list,sync_list,alone_list
print ("number of files is: ",len(spontan_list),',',len(sync_list),',',len(alone_list),
       "in spontan, sync and alone respectively")
#shapes of dfs
train_list = [spontan_list,sync_list,alone_list]
for list_ in train_list:
    for file in list_:
        print(file.shape)

number of files is:  9 , 9 , 9 in spontan, sync and alone respectively
(9054, 22)
(8998, 22)
(9050, 22)
(8937, 22)
(8826, 22)
(9054, 22)
(9054, 22)
(8773, 22)
(4375, 22)
(9052, 22)
(9054, 22)
(9039, 22)
(9050, 22)
(8395, 22)
(9046, 22)
(9054, 22)
(8829, 22)
(4500, 22)
(4526, 22)
(4575, 22)
(4482, 22)
(4527, 22)
(4931, 22)
(4526, 22)
(4468, 22)
(4537, 22)
(2250, 22)


In [98]:
#validation set
complete_data=(r'C:\Users\meirk\Desktop\final_project_files\extraFiles\Validation/')
spontan_list_val, sync_list_val ,alone_list_val = spontan_list,sync_list,alone_list
print ("number of files in is: ",len(spontan_list),',',len(sync_list),',',len(alone_list),
       "in spontan, sync and alone respectively")
#shapes of dfs
validation_list = [spontan_list_val, sync_list_val ,alone_list_val]
for list_ in validation_list:
    for file in list_:
        print(file.shape)

number of files in is:  9 , 9 , 9 in spontan, sync and alone respectively
(9054, 22)
(8998, 22)
(9050, 22)
(8937, 22)
(8826, 22)
(9054, 22)
(9054, 22)
(8773, 22)
(4375, 22)
(9052, 22)
(9054, 22)
(9039, 22)
(9050, 22)
(8395, 22)
(9046, 22)
(9054, 22)
(8829, 22)
(4500, 22)
(4526, 22)
(4575, 22)
(4482, 22)
(4527, 22)
(4931, 22)
(4526, 22)
(4468, 22)
(4537, 22)
(2250, 22)


We uploaded the data (both training and validation sets) into 3 lists, one for each scenario. Each list contains 9 files, a file for each participant.

In [62]:
#HandRight
right_hand = pd.read_csv(r'C:\Users\meirk\Desktop\final_project_files\extraFiles\HandRight.csv')
right_hand.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,128.2883,15478,right,1,15.67167,226.2064,20.26470,34.26665,-195.214700,-35.043020,...,-0.277800,46.88831,207.3548,86.23589,203.8337,49.33507,237.4553,0.0,0.206389,0.0
1,128.3048,15480,right,1,16.24119,222.9389,20.11781,36.16443,-180.093900,-4.052799,...,-0.277763,47.54861,205.6568,86.42250,203.9553,47.88272,238.4482,0.0,0.243050,0.0
2,128.3218,15482,right,1,16.72461,220.9709,19.94972,26.35298,-95.088190,-11.157620,...,-0.278792,48.15030,204.5182,86.35149,204.6231,45.35921,236.8544,0.0,0.281134,0.0
3,128.3384,15484,right,1,17.09016,219.7124,19.67679,19.46804,-61.755180,-21.303780,...,-0.279768,48.64175,203.9989,86.16056,204.9939,44.18692,236.0927,0.0,0.277691,0.0
4,128.3551,15486,right,1,17.46017,219.3652,19.13128,21.94964,-8.355943,-35.081790,...,-0.278629,48.93135,203.3777,85.59017,204.5952,42.68003,235.2896,0.0,0.283008,0.0


In [99]:
right_hand.shape

(4812, 22)

## preprocessing

###### Search for NaN values

In [88]:
nan_sum = 0
#train set
print("train set")
for list_ in train_list:
    for file in list_:
        nan_sum += file.isna().sum().sum()
print("number of NaN values in train set is: ",nan_sum)
#validation set
print("validation set")
validation_list = [spontan_list_val, sync_list_val ,alone_list_val]
for list_ in validation_list:
    for file in list_:
        nan_sum = file.isna().sum().sum()
print("number of NaN values in validation test is: ",nan_sum)
#right hand
print("right hand set")
print("number of NaN values in right hand set is: ",right_hand.isna().sum().sum())

train set
number of NaN values in train set is:  0
validation set
number of NaN values in validation test is:  0
right hand set
number of NaN values in right hand set is:  2304


###### converting 'Hand Type' to ones - right and zeros - left

In [65]:
def convert_to_1and0 (column):
    for index in range(len(column)):
        if column.loc[index] == 'right':
            column.loc[index] = 1
        else:
            column.loc[index] = 0

In [54]:
right_hand[' Hand Type'].shape

(4812,)

In [83]:
right_hand.isna().sum()

Time                 0
 Frame ID            0
 Hand Type           0
 # hands             0
 Position X          0
 Position Y          0
 Position Z          0
 Velocity X          0
 Velocity Y          0
 Velocity Z          0
 Pitch               0
 Roll                0
 Yaw                 0
 Wrist Pos X       384
 Wrist Pos Y       384
 Wrist Pos Z       384
 Elbow pos X       384
 Elbow Pos Y       384
 Elbow Pos Z       384
 Grab Strenth        0
 Grab Angle          0
 Pinch Strength      0
dtype: int64